# 1 - Swiyu Signup and Exploration

Ahmed: Copy 1.1 - Goals from presentation here as a table

Ahmed: synchronize with the other headings, so there is the same number of headings

Ahmed: change all swiyu to swiyu

Ahmed: syntax checking text

Short overview of what we'll be doing:

1. Installing and creating credential
2. Using the HoW credential
3. Inspect messages from logs
4. Check linkability of presentations

## 1.1 - Install the swiyu app

The swiyu project includes a wallet app that we use to store the credentials obtained from the the different issuers that adhere to the SWIYU issuer specification.  
The wallet will also be used to perform and provide verifications to verifiers that adhere to the SWIYU verifiers specification.

a.	https://play.google.com/store/apps/details?id=ch.admin.foitt.swiyu&hl=en&pli=1  
b.	https://apps.apple.com/ch/app/swiyu/id6737259614

### Discussion Points

- what is the main usage of the swiyu app?
- what security is given?

## 1.2 - Get a Beta Government E-ID

Ahmed: Introduction: it is a test ID, summary of steps

from: https://www.bcs.admin.ch/bcs-web/#/beta-id/create

### Discussion Points

- how does this work with the "real" E-ID?
- which data is stored where?

## 1.3 - Create an E-ID HoW Credential

Ahmed: create a short paragraph for introduction.

Go to https://eid.c4dt.org and navigate to the issuer section  
a.	The website will ask you to verify you identity first  
b.	then it will issue you a credential for attending the workshop  
c.  All data will be populated from the Beta ID credential you verified except the grade, you will get to set that yourself.  
d.	You will receive the credential on your wallet.

### Discussion Points

- how is the test ID linked to the HoW Credential?
- which messages are exchanged between the wallet and the issuer?
- how is the issuer configured and certified?
- how does the issuer verify the validity of the test ID?

## 1.4 - Verify your Credential

Ahmed: create a short paragraph for introduction.

Go to https://eid.c4dt.org/verifier/verify where you will verify your newly gained VC.

### Discussion Points

- which messages are exchanged between the wallet and the issuer?
- supposing the credential is unlinkable, under which conditions is the presentation/verification also unlinkable?

## 1.5 - Inspect Logs

Ahmed: create a short paragraph for introduction.

Go to https://eid.c4dt.org/logs where you will find the server logs that match your queries.  
Look  for the logs that match your VC presentation (the request that goes out from your wallet when you scan the QR code to verify your newly acquired VC.)

### Tips to find your logs

Find your specific VC presentation request in the logs. Hint: look for the:
    a.	Timestamp - when you made the request  
    b.	The request content is preceeded by `=== REQUEST BODY ===`
    c.  And the body is preceeded by `=== RESPONSE BODY ===`

### Discussion Points

- who can see these messages?
- what will an attacker-in-the middle see, e.g., in an unsecured WiFi connection?

## 1.6 - Analyze Logs

Ahmed: create a short paragraph for introduction.

1.	The presentation is URL-encoded, so decode it first using python xyz
2.	From the decoded request body, find the encoded VC presentation. 
3.	Use https://www.sdjwt.co/decode to decode the presentation.
4.  Identify the different types of information stored there.

### Discussion Points

- Which parts do you think can be used to link two presentations?
- Look how the selective disclosure works when disclosing different parts of your credential
  - How is the data secured?
  - Why isn't it possible to guess low-entropy entries like the postal code?

In [ ]:
# Use a python base64 decoder of the data to show the main representation

# Ahmed: write 'import' and 'print(base64.decode("abcd"))`

## 1.7 - Linkability of Multiple Presentations

Ahmed: create a short paragraph for introduction. E.g., let's look how two presentations from an unlinkable 
field like the grade can still be linked to the same holder.

Do two verifications of the HoW credential, fetch the presentation from the Logs, and search for fixed strings.

### Discussion Points

- What can make it very hard to avoid linkability when you think about the fields you're sharing with the Verifier?
- Which fields can be revealed and keep anonymity?
- Check with your neighbours to see which fields are constant between two wallets
- What is a simple way to remove some of the linkability?